# **자기주도PJT - 심화과제**

**Req.7. 다음의 과정을 따라 advanced.ipynb에 VoxCeleb Dataset을 사용해 결과를 출력 해보세요.**
1. https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw 에서 동영상 파일 (예, 04.mp4) 하나를 다운로드 받아 구글 드라이브 first_motion_order_model 폴더에 복사해 놓습니다.
2. https://drive.google.com/drive/folders/1PyQJmkdCsAkOYwUyaj_l-l0as-iLDgeH 에서 다음의 Dataset 파일을 다운로드 받아 구글 드라이브 first_motion_order_model 폴더에 복사해 놓습니다.
  - vox-cpk.pth.tar
3. 휴대폰으로 셀카를 찍어 my_face.png 라는 파일명으로 구귿 드라이브 first_motion_order_model 폴더에 올려 놓습니다. (정사각형 형태로 촬영을 추천 합니다.)


In [ ]:
# 구글 드라이브를 /gdrive에 마운트 합니다.
from google.colab import drive
drive.flush_and_unmount()  # 파일목록 갱신이 안될때는 이 부분 주석처리를 해제하고 실행하시기 바랍니다.
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# 마운트된 드라이브의 first_order_motion_model 디렉토리의 파일목록을 불러옵니다.
!ls -l /gdrive/MyDrive/Colab/first_motion_order_model

total 712727
-rw------- 1 root root    490875 Jun 17 04:51 04.mp4
-rw------- 1 root root      7668 Jun 17 05:06 advanced.ipynb
-rw------- 1 root root    566729 Jun 17 04:58 my_face.jpg
-rw------- 1 root root 728766691 Jun 17 04:57 vox-cpk.pth.tar


In [ ]:
# Aliaksandr Siarohin 의 github에서 First Order Model을 clone 합니다.
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 299, done.
remote: Total 299 (delta 0), reused 0 (delta 0), pack-reused 299
Receiving objects: 100% (299/299), 72.15 MiB | 36.48 MiB/s, done.
Resolving deltas: 100% (154/154), done.


In [ ]:
# 디렉토리를 변경합니다.

In [ ]:
cd /content/first-order-model

/content/first-order-model


In [ ]:
# 현재 디렉토리의 내용을 보여줍니다.
!ls -l

total 1276
-rw-r--r-- 1 root root    4061 Jun 17 05:07 animate.py
-rw-r--r-- 1 root root   12547 Jun 17 05:07 augmentation.py
drwxr-xr-x 2 root root    4096 Jun 17 05:07 config
-rw-r--r-- 1 root root    5876 Jun 17 05:07 crop-video.py
drwxr-xr-x 3 root root    4096 Jun 17 05:07 data
-rw-r--r-- 1 root root   21284 Jun 17 05:07 demo.ipynb
-rw-r--r-- 1 root root    6730 Jun 17 05:07 demo.py
-rw-r--r-- 1 root root     502 Jun 17 05:07 Dockerfile
-rw-r--r-- 1 root root    7027 Jun 17 05:07 frames_dataset.py
-rw-r--r-- 1 root root   18868 Jun 17 05:07 LICENSE.md
-rw-r--r-- 1 root root    8282 Jun 17 05:07 logger.py
drwxr-xr-x 2 root root    4096 Jun 17 05:07 modules
-rw-r--r-- 1 root root 1143882 Jun 17 05:07 old_demo.ipynb
-rw-r--r-- 1 root root    8727 Jun 17 05:07 README.md
-rw-r--r-- 1 root root    2772 Jun 17 05:07 reconstruction.py
-rw-r--r-- 1 root root     210 Jun 17 05:07 requirements.txt
-rw-r--r-- 1 root root    3299 Jun 17 05:07 run.py
drwxr-xr-x 2 root root    4096 Jun 17 05:07 

In [ ]:
# 필요한 패키지들을 import 합니다.
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte


In [ ]:
# 소스 이미지
source_image = imageio.imread('/gdrive/MyDrive/Colab/first_motion_order_model/my_face.jpg')
# 소스 비디오
reader = imageio.get_reader('/gdrive/MyDrive/Colab/first_motion_order_model/04.mp4')
# 이미지와 비디오크기를 256x256 픽셀로 변경 합니다
source_image = resize(source_image, (256, 256))[..., :3]
# frames per second 1초당 표시할 프레임수를 가져옵니다
fps = reader.get_meta_data()['fps']

In [ ]:
#driving_video를 초기화 하고 소스비디오의 각 프레임을 driving_video에 추가 합니다.
driving_video = []
try:
  for im in reader:
    driving_video.append(im)
except RuntimeError:
  pass
reader.close()

In [ ]:
# driving_video를 256x256 으로 리사이즈 합니다.
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

In [ ]:
# 소스이미지와 기존 비디오를 합성해주는 display 함수
def display(source, driving, generated=None):
  fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
  ims = []
  for i in range(len(driving)):
    cols = [source]
    cols.append(driving[i])
    if generated is not None:
      cols.append(generated[i])
    im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
    plt.axis('off')
    ims.append([im])
  
  ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
  plt.close()
  return ani

In [ ]:
# display() 함수를 사용하여 HTML5 비디오 형태로 출력합니다.
HTML(display(source_image, driving_video).to_html5_video())

In [ ]:
# VoxCeleb 모델의 체크포인트 불러오기 (700MB 파일이라 오래 걸립니다.)
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml',
  checkpoint_path='/gdrive/MyDrive/Colab/first_motion_order_model/vox-cpk.pth.tar')

In [ ]:
# make_animation() 함수를 사용하여 소스이미지, 소스비디오를 입력으로 predictions를 생성합니다.
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

# generated.mp4 파일로 생성된 결과를 저장
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)

# display 함수를 사용해 로 출력합니다.
HTML(display(source_image, driving_video, predictions).to_html5_video())



100%|██████████| 211/211 [00:07<00:00, 26.59it/s]






---




**Req.8. 다음의 과정을 따라 advanced.ipynb에 Fashion-Videos Dataset을 사용해 결과를 출력 해보세요.**
1.	https://drive.google.com/drive/folders/1PyQJmkdCsAkOYwUyaj_l-l0as-iLDgeH 에서 다음의 Dataset 파일을 다운로드 받아 구글 드라이브 first_motion_order_model 폴더에 복사해 놓습니다.
  -	fashion.pth.tar
2.	https://vision.cs.ubc.ca/datasets/fashion/resources/fashion_dataset/fashion_train.txt 에서 4개의 mp4 파일을 다운로드 받습니다. 
3.	다운로드 받은 mp4파일을 전체화면으로 재생시킨 후 바로 Pause 버튼을 눌러 모델이 똑바로 앞을 보고 있는 장면에서 멈춘 후 컴퓨터의 화면 캡춰 프로그램을 사용해 동영상 화면을 캡춰해 m1,png, m2,png, mp3.png 등 3개의 정지화면을 얻은 후 구글드라이브에 복사해 놓습니다.
4.	나머지 1개의 동영상 혹은 해당 동영상과 비슷한 구도와 포즈의 전신 샷을 촬영하여 req8_source.mp4라는 이름으로 구글드라이브에 복사해 놓습니다.


In [ ]:
from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte

In [28]:
from imageio.core.functions import mimread
# 3개의 이미지와 1개의 동영상을 불러 옵니다.
src_img1 = imageio.imread('/gdrive/MyDrive/Colab/first_motion_order_model/m1.jpg')
src_img2 = imageio.imread('/gdrive/MyDrive/Colab/first_motion_order_model/m2.jpg')
src_img3 = imageio.imread('/gdrive/MyDrive/Colab/first_motion_order_model/m3.jpg')

driving_video = imageio.mimread('/gdrive/MyDrive/Colab/first_motion_order_model/req8_source.mp4', memtest=False)

In [29]:
# 이미지와 비디오를 256x256 사이즈로 변경 합니다.
src_img1 = resize(src_img1, (256, 256))[..., :3]
src_img2 = resize(src_img2, (256, 256))[..., :3]
src_img3 = resize(src_img3, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

In [31]:
# load_checkpoints 함수를 사용해 이전에 학습해둔 fashion.pht.tar을 을 불러 옵니다.
ggnerator, kp_detector = load_checkpoints(config_path='config/fashion-256.yaml',
                                          checkpoint_path='/gdrive/MyDrive/Colab/first_motion_order_model/fashion.pth.tar')

In [32]:
# 첫번째 모델의 사진과 Driving Video로 첫번째 모델의 움직이는 비디오를 만들고 재생합니다.
predictions1 = make_animation(src_img1, driving_video, generator, kp_detector, relative=True)
HTML(display(src_img1, driving_video, predictions1).to_html5_video())

100%|██████████| 422/422 [00:17<00:00, 24.26it/s]


In [33]:
# 두번째 모델의 사진과 Driving Video로 두번째 모델의 움직이는 비디오를 만들고 재생합니다.
predictions2 = make_animation(src_img2, driving_video, generator, kp_detector, relative=True)
HTML(display(src_img2, driving_video, predictions2).to_html5_video())

100%|██████████| 422/422 [00:16<00:00, 25.51it/s]


In [34]:
# 세번째 모델의 사진과 Driving Video로 세번째 모델의 움직이는 비디오를 만들고 재생합니다.
predictions3 = make_animation(src_img3, driving_video, generator, kp_detector, relative=True)
HTML(display(src_img3, driving_video, predictions3).to_html5_video())

100%|██████████| 422/422 [00:16<00:00, 25.54it/s]
